In [ ]:

# 📊 IA Café Commodity - Análise de Mercado + Estratégia Personalizada com Margem de Lucro

# Etapa 1: Instalar bibliotecas
!pip install yfinance ipywidgets --quiet


In [ ]:

# Etapa 2: Importar bibliotecas
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:

# Etapa 3: Coletar dados do Café Contrato C
dados = yf.download("KC=F", start="2020-01-01")

# Calcular médias móveis
dados['SMA_10'] = dados['Close'].rolling(window=10).mean()
dados['SMA_30'] = dados['Close'].rolling(window=30).mean()

# Gerar sinais
dados['Sinal'] = 0
dados.loc[dados['SMA_10'] > dados['SMA_30'], 'Sinal'] = 1
dados.loc[dados['SMA_10'] < dados['SMA_30'], 'Sinal'] = -1

# Gráfico com médias móveis
dados[['Close', 'SMA_10', 'SMA_30']].plot(figsize=(12, 6), grid=True)
plt.title("📈 Preço do Café Contrato C com Médias Móveis")
plt.xlabel("Data")
plt.ylabel("Preço (USD)")
plt.show()

# Mostrar último sinal
ultimo_sinal = dados['Sinal'].iloc[-1]
print("📍 Último sinal de operação:", "🔼 COMPRA" if ultimo_sinal == 1 else "🔽 VENDA" if ultimo_sinal == -1 else "⏸️ NEUTRO")


In [ ]:

# Etapa 4: Formulário de Estoque + Estratégia com Margem de Lucro

# Widgets para entrada de dados
volume_estoque = widgets.FloatText(description='Estoque (sacas):', value=1000)
custo_medio = widgets.FloatText(description='Custo médio (R$/saca):', value=750)
margem_desejada = widgets.FloatText(description='Margem desejada (R$):', value=150)
botao_estoque = widgets.Button(description='Analisar Estratégia')
saida_estoque = widgets.Output()


In [ ]:

# Função de análise estratégica
def analisar_estoque(b):
    with saida_estoque:
        clear_output()
        print("🔍 Analisando mercado e estoque...")

        dados_atuais = yf.download("KC=F", period="7d", interval="1d")

        if not dados_atuais.empty and not dados_atuais['Close'].isna().all():
            preco_dolar = dados_atuais['Close'].dropna().iloc[-1]
            preco_atual = preco_dolar * 5.5  # conversão estimada

            meta_venda = custo_medio.value + margem_desejada.value
            margem = preco_atual - custo_medio.value

            print(f"📦 Volume atual: {volume_estoque.value} sacas")
            print(f"💲 Custo médio: R${custo_medio.value:.2f}")
            print(f"📈 Margem desejada: R${margem_desejada.value:.2f}")
            print(f"🎯 Meta de venda (calculada): R${meta_venda:.2f}")
            print(f"📉 Preço atual estimado: R${preco_atual:.2f}")
            print(f"🧾 Margem atual: R${margem:.2f}")

            if preco_atual >= meta_venda:
                print("\n✅ Recomendação: Meta de lucro atingida! Avaliar venda parcial ou total.")
            elif margem > 0:
                print("\n🔄 Recomendação: Está no lucro, mas abaixo da margem esperada. Considere hedge ou venda parcial.")
            else:
                print("\n⏳ Recomendação: Preço abaixo do custo. Melhor segurar por enquanto.")
        else:
            print("❌ Não foi possível obter a cotação atual do café. Tente novamente em alguns minutos.")


In [ ]:

# Exibir formulário
botao_estoque.on_click(analisar_estoque)
display(volume_estoque, custo_medio, margem_desejada, botao_estoque, saida_estoque)
